In [ ]:
import gen3_validator
# loading gen3 bundled jsonschema into dictionary class
dd = gen3_validator.dict.DataDictionary('../examples/schema/json/schema_dev.json')
dd.parse_schema()

In [5]:
# getting list of entities
dd.get_nodes()

['demographic.yaml',
 'project.yaml',
 'acknowledgement.yaml',
 'medical_history.yaml',
 '_definitions.yaml',
 '_settings.yaml',
 'program.yaml',
 'sample.yaml',
 '_terms.yaml',
 'lipidomics_file.yaml',
 'publication.yaml',
 'subject.yaml',
 'core_metadata_collection.yaml']

In [6]:
# Returning the jsonschema for a given entity
dd.return_schema('demographic.yaml')

{'$schema': 'http://json-schema.org/draft-04/schema#',
 'id': 'demographic',
 'title': 'Demographic',
 'type': 'object',
 'namespace': 'https://data.acdc.ozheart.org',
 'category': 'clinical',
 'program': '*',
 'project': '*',
 'description': 'Data for the characterization of the patient by means of segementing the population (e.g. characterization by age, sex, or race).',
 'additionalProperties': False,
 'submittable': True,
 'validators': None,
 'systemProperties': ['id',
  'project_id',
  'state',
  'created_datetime',
  'updated_datetime'],
 'links': [{'name': 'subjects',
   'backref': 'demographics',
   'label': 'describes',
   'target_type': 'subject',
   'multiplicity': 'one_to_one',
   'required': True}],
 'required': ['type', 'submitter_id', 'subjects'],
 'uniqueKeys': [['id'], ['project_id', 'submitter_id']],
 'properties': {'$ref': '_definitions.yaml#/ubiquitous_properties',
  'subjects': {'$ref': '_definitions.yaml#/to_one'},
  'year_birth': {'description': 'Year of birth a